Обозначения:

📝 - определение, которое нужно запомнить

`# ⏩`- комментарий, что эту ячейку / функцию нужно просто запустить, она уже написана

`# ✏️` - код в этой ячейке / функции мы будем дополнять в процессе урока

👨🏻‍💻 - задача для самостоятельного выполнения

[Чистая рабочая тетрадь для заполнения](https://colab.research.google.com/drive/1hQ69Nn3nnCF4PpBN2HOxXXDJQQm-mL0i?usp=sharing)

[Заполненная рабочая тетрадь](https://colab.research.google.com/drive/1DOIDJXYM9a-DAGGnHIK4qyZECwg0higW?usp=sharing)

[Презентация к уроку](https://docs.google.com/presentation/d/10m7FYPHtHWrby_vl6Z1a-pkrL8magcM0wIruskHNka4/edit?usp=sharing)

# Чем мы будем заниматься на уроке?

Мы напишем систему для подбора похожих новостей. Возьмем данные о новостях с Lenta.ru, с помощью нейросети Word2Vec превратим текстовые описания в числа и объединим в группы на основе близости чисел.

# Немного теории: концептуальная схема работы нейронной сети

Вспомним, в чем заключается задача Data Scientist:
<table>
  <tr>
    <th></th> <th>Прямая задача</th> <th>Обратная задача</th>
  </tr>
  <tr>
    <th>Дано</th>
    <td>y = f(x)</td>
    <td bgcolor="#93E9BE">Координаты y</td>
  </tr>
  <tr>
    <th>Найти</th>
    <td>Координаты y</td>
    <td bgcolor="#93E9BE">Функцию y = z(x)</td>
  </tr>
  <tr>
    <th></th>
    <th>↑ Этому учат в школе ↑</th>
    <th>↑ <u>Этим занимаются DS</u>↑</th>
  </tr>
</table>

📝 **Нейронная сеть** - тип модели машинного обучения, которая состоит из комбинации нейронов - простых функций вида $y_k = w_1 x_2 + ... + w_i x_i$.

📝 **Веса модели** - коэффициенты $w_1 , \dots,  x_i$

Крутость нейросети в том, с помощью нейронов мы можем приблизительно вычислить практически любую функцию (аналогия: цифровое фото из квадратных пикселей). Любая модель семейства GPT по сути являются функцией, описывающей текст на человеческом языке, т.н. "большая языковая модель".

Один из простейших примеров - вычисление функции XOR(a, b) - исключающего ИЛИ. Аргументы a и b могут быть равны 1 или 0. XOR(a, b) вычисляется по следующему правилу:
* XOR = 0, если a = b,
* XOR = 1, если a != b.

Нейронная сеть будет вычислять эту функцию по следующей схеме:

<table style="border: 1px solid transparent">
    <tr><td><img src='https://drive.google.com/uc?export=view&id=1zn68tWxZpZzWOrtxlL4_MHy8MWFIyrEE' height=200>
    </td></tr>
    <tr><td>Картинка с <a href="https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD">Википедии</a></td></tr>
</table>


Сложность работы с нейронной сетью - это подбор правильных $w_i$. Для этого используются разные комбинации нейронов и разные типы задач.

Например, чтобы преобразовать текст в числа, модель обучают угадывать замаскированное слово по контексту, а потом берут получившиеся веса.
Например, "Мороз и солнце, [???] чудесный". После получения представления текста в виде чисел из весов модели, мы можем выполнять различные математические преобразования над текстом, например "король − мужчина + женщина = королева" (можно подробнее почитать на [Хабре](https://habr.com/ru/articles/446530/))

# Практика - подбираем релевантные новости с помощью нейросети

Пишем систему для рекомендации похожих новостей.

Наш датасет с новостями имеет темы - **topic** ("Спорт", "Наука и техника", ...) и тэги **tag** ("Футбол", "Техника", "Космос", ...).

1. Возьмем новости, помеченные одним тэгом и разобьем на еще более маленькие группы по похожести текстов.

2. Используем библиотеку, реализующую нейросеть архитектуры Word2Vec от [gensim](https://pypi.org/project/gensim/) и готовые веса для нее от [RusVectores](https://rusvectores.org/ru/models/). Преобразуем тексты в наборы чисел.

3. Используем модели для 📝 **кластеризации** - разбиения набора данных на группы наиболее похожих элементов. Воспользуемся реализацией из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [1]:
# ▶️▶️
# Подключаем нужные библиотеки для визуализации

import nltk
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans

nltk.download('punkt') # загружаем дополнительные данные для библиотеки разбиения текстов на слова

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# ▶️▶️
# Настраиваем отображение таблиц
from IPython.display import display
pd.set_option('display.max_colwidth', 200)

In [3]:
# ▶️▶️
# Скачиваем библиотеку для загрузки файлов
!pip install wget --quiet

  Preparing metadata (setup.py) ... done


Команда %%time в начале ячейки замеряет время ее выполнения

In [4]:
%%time
# ▶️▶️
# Выполняется ≈1 минуту

# Скачиваем веса модели

from wget import download

download('https://rusvectores.org/static/models/rusvectores4/news/news_upos_cbow_600_2_2018.vec.gz',
         'news_upos_cbow_600_2_2018.vec.gz')

CPU times: user 2.25 s, sys: 1.37 s, total: 3.62 s
Wall time: 32.2 s


'news_upos_cbow_600_2_2018.vec.gz'

In [5]:
%%time
# ▶️▶️
# Выполняется ≈2-3 минуты
word2vec = KeyedVectors.load_word2vec_format('news_upos_cbow_600_2_2018.vec.gz')

CPU times: user 2min 13s, sys: 1.42 s, total: 2min 14s
Wall time: 2min 17s


In [ ]:
# ✏️
# Атрибут index_to_key объекта word2vec содержит список слов,
# на котором обучалась модель

word2vec.index_to_key

У нашей модели есть особенность: для ее обучения к словам цепляли [тэги с частью речи](https://universaldependencies.org/u/pos/all.html), например, 'год_NOUN', 'сообщать_VERB'.

Чтобы сопоставить обычное слово и слово с тэгом из модели, составим словарь переименования.

In [7]:
# ▶️▶️
# Метод split разбивает слово на список подслов по заданному символу

'год_NOUN'.split('_')

['год', 'NOUN']

In [8]:
# ▶️▶️
# В квадратных скобках из списка получаем начальный элемент (нумерация с 0)
'год_NOUN'.split('_')[0]

'год'

In [12]:
# ✏️
# Создадим словарь вида {слово: слово_ТЭГ}.
# В конце списка встречаются некорректные тэги, например, 'год_NOUN' 'год_PROPN'
# или ['давать_VERB', 'давать_NOUN', 'давать_ADJ', 'давать_PROPN', 'давать_NUM']
# Поэтому будем брать только первую версию слова, и если оно уже есть в словаре,
# перезаписывать его не будем

lemma2word = {}

for tagged_word in word2vec.index_to_key:
    word = tagged_word.split('_')[0]
    if word not in lemma2word:
        lemma2word[word] = tagged_word

In [15]:
# ▶️▶️
lemma2word['дать']

'дать_NOUN'

# Векторизация

Для того, чтобы иметь возможность автоматически анализировать данные, нужно придать тексту числовую структуру. Для этого сделаем векторизацию. Нам нужно пройти несколько стадий: токенизация → лемматизация → векторизация.

📝 **Токенизация** - процесс разбиения текста на слова (**токен** - минимальная единица язчка, имеющая смысл)

👨🏻‍💻 Вопрос: можно ли назвать букву токеном?

📝 **Лемматизация** - получение начальной формы слова: "годы", "году" → "год"

📝 **Векторизация** - процесс превращения текста в набор чисел - **векторное представление**. Например, "год" → (0.1, 0, 1.2, 3)

📝 **Вектор** - набор чисел, например, (0.1, 0, 1.2, 3). Над векторами можно проводить арифметические операции аналогично числам - например, попарное сложение элементов: (1, 2, 5) + (3, 4, 0) = (4, 6, 5) или вычисление расстояния между ними.

На основе близости векторных представлений текстов мы и будем искать похожие новости.

In [16]:
# ▶️▶️
# Пример токенизации

word_tokenize('Мороз и солнце, день чудесный', language='russian')

['Мороз', 'и', 'солнце', ',', 'день', 'чудесный']

In [17]:
# ▶️▶️
# mystem - инструмент для лемматизации
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [18]:
# ▶️▶️
# Функция получения тэгированной формы слова из модели Word2Vec

def get_w2v_word(word):
    # Получаем начальную форму слова
    lemma = mystem.lemmatize(word)[0]
    # Берем тэгированную форму из составленного ранее словаря
    w2v_word = lemma2word.get(lemma)
    return w2v_word

In [19]:
# ▶️▶️
get_w2v_word("Мороз")

'мороз_NOUN'

In [20]:
get_w2v_word("даю")

'давать_VERB'

In [21]:
# ▶️▶️
# Функция получения векторного представления слова из модели Word2Vec

def vectorize_sentence(txt):
    words = word_tokenize(txt, language='russian')
    vectors = []
    for word in words:
        tagged_word = get_w2v_word(word)
        if tagged_word is not None:
            vector = word2vec[tagged_word]
            vectors.append(vector)
    return np.mean(vectors, axis=0)

In [ ]:
# ▶️▶️
vectorize_sentence('Мороз и солнце, день чудесный')

# Возьмем данные о новостях с Lenta.ru

In [23]:
# ▶️▶️
data = pd.read_csv('https://raw.githubusercontent.com/anastasiarazb/skillbox_nlp_demo/master/lenta_example.csv',
                   sep=',')
data

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины российских биатлонистов,Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встреч...,Спорт,Зимние виды
1,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти на «сатанинское сборище» в Киеве,Архиепископ канонической Украинской православной церкви Московского патриархата (УПЦ МП) Лонгин отказался участвовать в «объединительном соборе» по приглашению Вселенского патриарха Варфоломея. Он...,Бывший СССР,Украина
2,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал хет-трик в игре Национальной хоккейной лиги (НХЛ) с «Каролиной». Таким образом он продлил результативную серию до 13 матчей и повторил личный рекорд ...,Спорт,Хоккей
3,https://lenta.ru/news/2018/12/15/newyear/,Названы регионы России с самым дорогим и дешевым новогодним столом,"Производитель онлайн-касс «Эвотор» проанализировал данные со своих терминалов, установленных в тысячах продуктовых магазинов по стране, и рассчитал стоимость базового новогоднего стола в разных го...",Экономика,Деньги
4,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,"Вице-президент Еврокомиссии Марош Шефчович пригласил Россию и Украину в Брюссель для следующего этапа трехсторонних переговоров по газу. Встреча состоится в январе, написал он в Twitter. «Ожидаем,...",Экономика,Госэкономика
...,...,...,...,...,...
24462,https://lenta.ru/news/2017/10/27/rynoktruda/,Десяти миллионам россиян предложили сменить профессию,"Россию ждет дефицит специалистов, способных выполнять аналитическую и творческую работу. К 2025 году нехватка таких кадров может достигнуть 10 миллионов человек. При этом столько же людей могут ок...",Экономика,Социальная сфера
24463,https://lenta.ru/news/2017/10/27/singsing/,Электромобили Nissan запоют ради пешеходов,"Японская автомобильная компания Nissan продемонстрировала на Токийском международном автосалоне звуковую систему Canto, разработанную специально для электромобилей. Видеопрезентация опубликована н...",Ценности,Движение
24464,https://lenta.ru/news/2017/10/27/red_russian/,Швейцарцы выпустили часы за 20 тысяч долларов в честь Октябрьской революции,"Швейцарская часовая компания H. Moser & Cie. представила новую модель часов Venturer XL Stoletniy Krasniy, сообщает Monochrome. Дизайн модели с темнеющим по краям красным циферблатом, надписью «Г....",Ценности,Часы
24465,https://lenta.ru/news/2017/10/27/crispr/,Создан эффективный способ лечения смертоносных мутаций,"Ученые из Массачусетского технологического института и Гарвардского университета разработали новый способ регулировки активности генов, который позволит лечить наследственные заболевания. Метод ос...",Наука и техника,Наука


In [24]:
# ✏️
# Выведем списки тем и тэгов со счетчиком числа статей в этих группах
# Функция groupby по списку колонок ['topic', 'tags'] группирует данные,
# а count() - дает количество записей в каждой группе

data.groupby(['topic', 'tags']).count()

url  title  text
topic           tags                               
Бывший СССР     undefined           46     46    46
                Белоруссия         162    162   162
                Все                  4      4     4
                Закавказье         245    245   245
                Молдавия            36     36    36
                Прибалтика         102    102   102
                Средняя Азия       117    117   117
                Украина           2548   2548  2548
Дом             Город              356    356   356
                Дача               236    236   236
                Квартира          1371   1371  1371
                Офис               125    125   125
Культура        undefined            2      2     2
                Все                  8      8     8
                Искусство          135    135   135
                Кино              1486   1486  1486
                Книги              124    124   124
                Музыка            1229   1229  1229
                Театр              171    171   171
                Фотография          16     16    16
Наука и техника Все                 11     11    11
                Гаджеты            198    198   198
                Жизнь              286    286   286
                Игры                35     35    35
                История            113    113   113
                Космос             699    699   699
                Наука              820    820   820
                Оружие            1144   1144  1144
                Софт                22     22    22
                Техника            183    183   183
Спорт           undefined          546    546   546
                Авто                62     62    62
                Бокс и ММА         682    682   682
                Все                 18     18    18
                Зимние виды        816    816   816
                Летние виды        247    247   247
                Футбол            2798   2798  2798
                Хоккей             289    289   289
Ценности        undefined            8      8     8
                Вкусы               37     37    37
                Внешний вид        310    310   310
                Все                  1      1     1
                Движение           150    150   150
                Инструменты        127    127   127
                Стиль              571    571   571
                Часы               135    135   135
                Явления            725    725   725
Экономика       undefined          199    199   199
                Бизнес             880    880   880
                Все                 40     40    40
                Госэкономика      2390   2390  2390
                Деловой климат     562    562   562
                Деньги             273    273   273
                Рынки              505    505   505
                Социальная сфера    66     66    66

In [36]:
# ✏️
# В теме 'Экономика' возьмем тэг 'undefined' и исследуем новости в нем

data_tag = data[
    (data['topic'] == 'Наука и техника')
    & (data['tags'] == 'Софт')
    ].copy()

# 👨🏻‍💻 Попробуйте другую комбинацию темы и тэга, напр. "Наука и техника"+"Техника"

data_tag.shape

(22, 5)

In [37]:
# ✏️

# Представим заголовки и тексты в виде чисел с помощью
# объединения векторных представлений, полученных с помощью
# vectorize_sentence для каждого в отдельности
# Операция + для двух списков объединяет их: [1, 2] + [3, 4] = [1, 2, 3, 4]

vectors = []

for title, text in tqdm(data_tag[['title', 'text']].values):
    vector = vectorize_sentence(title) + vectorize_sentence(text)
    vectors.append(vector)

# 👨🏻‍💻 Попробуйте использовать только 1 элемент статьи, только текст
#    или только заголовок
#    (для этого в цикле надо будет оставить в vector, например,
#    только vectorize_sentence(text))

vectors = np.array(vectors)

  0%|          | 0/22 [00:00<?, ?it/s]

# Кластеризация

📝 **Кластеризация** - объединение похожих объектов в группы.

Есть различные методы кластеризации, мы будем пользоваться библиотекой [sklearn](https://scikit-learn.org/stable/modules/clustering.html)

In [38]:
# ▶️▶️

from sklearn.cluster import SpectralClustering, KMeans

clusterizer = SpectralClustering(n_clusters=8, n_init=10, random_state=42)

# 👨🏻‍💻 Попробуйте другую модель кластеризации, например, KMeans.
#    Описание различных методов: https://scikit-learn.org/stable/modules/clustering.html

# 👨🏻‍💻 Попробуйте заменить параметры, например, установить n_clusters=10.
#    Есть ли какие-то идеи, как правильно выбрать количество кластеров?

clusterizer.fit(vectors)

SpectralClustering(random_state=42)

In [39]:
# ✏️
# Добавим разметку с номером кластера clusterizer.labels_
# как новую колонку в данные 'label' в таблицу data_tag
data_tag['label'] = clusterizer.labels_

In [40]:
data_tag.head(5)

,url,title,text,topic,tags,label
789,https://lenta.ru/news/2018/12/04/edge_rip/,Microsoft похоронит свой браузер,Корпорация Microsoft занимается разработкой нового браузера на основе платформы Chromium от Google. Об этом пишет Windows Central. Проект нового браузера имеет кодовое название Anaheim и должен за...,Наука и техника,Софт,3
4152,https://lenta.ru/news/2018/10/09/myoffice/,Российский софт оказался ущербным,"В департаменте информационных технологий (ДИТ) Москвы пожаловались на дефекты приложения «МойОфис Почта», разработанного компанией «Новые облачные технологии» (НОТ). Об этом пишет «Коммерсантъ». В...",Наука и техника,Софт,0
5281,https://lenta.ru/news/2018/09/20/waitlist/,Windows уличили в сборе паролей и переписки,"В операционной системе Windows нашли функцию, которая сохраняет и помещает в определенный файл пароли и переписку пользователей. Об этом сообщает ZDNet. Отмечается, что функция по сбору данных пол...",Наука и техника,Софт,5
5487,https://lenta.ru/news/2018/09/17/bug/,Найден новый способ «убить» iPhone,"Специалист по информационной безопасности Сабри Хэддоуч (Sabri Haddouche) нашел новую уязвимость, позволяющую отключить любой iOS-девайс. Об этом он написал в Twitter. Ошибка кроется в CSS-коде и ...",Наука и техника,Софт,2
8269,https://lenta.ru/news/2018/07/31/amigo/,Mail.ru похоронил браузер «Амиго»,"Компания Mail.ru Group объявила о прекращении разработки и продвижения браузера «Амиго». Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру». В сообщении указывается, что при прод...",Наука и техника,Софт,3


In [41]:
# ✏️

# Набор меток data_tag['label'].unique() отсортируем через sorted()
# и для каждой метки выведем label и подтаблицу строк с этой
# меткой кластера data_tag[data_tag['label'] == label]

for label in sorted(data_tag['label'].unique()):
    print(label)
    display(data_tag[data_tag['label'] == label])

0


,url,title,text,topic,tags,label
4152,https://lenta.ru/news/2018/10/09/myoffice/,Российский софт оказался ущербным,"В департаменте информационных технологий (ДИТ) Москвы пожаловались на дефекты приложения «МойОфис Почта», разработанного компанией «Новые облачные технологии» (НОТ). Об этом пишет «Коммерсантъ». В...",Наука и техника,Софт,0
8420,https://lenta.ru/news/2018/07/27/russian_plan/,Россиянам навяжут отечественный софт в гаджетах,"Федеральная антимонопольная служба (ФАС) внесла в правительство инициативу, которая обяжет оснащать гаджеты приложениями российского производства. Об этом сообщает «Коммерсантъ» со ссылкой на «дор...",Наука и техника,Софт,0
13555,https://lenta.ru/news/2018/04/29/alternative_android/,В Китае придумали замену Android на случай американских санкций,"Китайский производитель смартфонов Huawei работает над собственной операционной системой (мобильной и десктопной) для замены Android и Windows, пишет South China Morning Post (SCMP) со ссылкой на ...",Наука и техника,Софт,0
14436,https://lenta.ru/news/2018/04/15/operavpn/,Opera VPN объявила о скором прекращении работы,"Компания Opera Software объявила о прекращении работы приложения Opera VPN с 30 апреля 2018 года. Сообщение было опубликовано на официальном сайте компании в воскресенье, 15 апреля. «Мы прекращаем...",Наука и техника,Софт,0
20397,https://lenta.ru/news/2018/01/02/fuchsia/,Google разрешила тестировать таинственную замену Android,"Американская корпорация Google разрешила разработчикам начать тестирование новой операционной системы Fuchsia, пишет Chrome Unboxed. Компания выложила документацию, позволяющую загрузить ОС на Pix...",Наука и техника,Софт,0


1


,url,title,text,topic,tags,label
14368,https://lenta.ru/news/2018/04/16/utorrent/,Microsoft заблокировала uTorrent,"Встроенный антивирус Windows Defender начал блокировать работу торрент-клиента uTorrent, обозначая программу как вирус. Об этом пишет портал ExtremeTech. На сайте Microsoft еще в феврале 2017 года...",Наука и техника,Софт,1


2


...

,url,title,text,topic,tags,label
5281,https://lenta.ru/news/2018/09/20/waitlist/,Windows уличили в сборе паролей и переписки,"В операционной системе Windows нашли функцию, которая сохраняет и помещает в определенный файл пароли и переписку пользователей. Об этом сообщает ZDNet. Отмечается, что функция по сбору данных пол...",Наука и техника,Софт,5
17619,https://lenta.ru/news/2018/02/19/edge/,Google раскрыла опасную уязвимость Windows,"Разработчики Google в рамках проекта Project Zero обнаружили уязвимость в браузере Microsoft Edge. Долгое время компания не сообщала об уязвимости, однако в Microsoft не успели ее исправить, пишет...",Наука и техника,Софт,5
21678,https://lenta.ru/news/2017/12/10/virus/,Программисты создали необнаруживаемый вирус для Windows,"Работники компании enSilo, представили новейший метод, позволяющий обойти все антивирусные программы. Разработка была представлена на Black Hat Europe 2017, сообщает издание Techspot. Антивирусные...",Наука и техника,Софт,5


6


,url,title,text,topic,tags,label
13189,https://lenta.ru/news/2018/05/08/morse/,Google научился морзянке,"Google добавила азбуку Морзе в клавиатуру Gboard, опцию представили на ежегодной конференции для разработчиков, трансляция шла на сайте мероприятия. Клавиатура с морзянкой представляет собой два п...",Наука и техника,Софт,6


7


,url,title,text,topic,tags,label
10831,https://lenta.ru/news/2018/06/18/excel/,Россиянин дошел до конца Excel,"Пользователь Twitter под ником Отец Golem вручную заполнил все 17 179 869 184 ячейки Excel. На это у него ушло два года. Он подчеркивает, что все сделал вручную без использования скриптов. Чтобы з...",Наука и техника,Софт,7


# Что мы узнали на этом занятии?

* ML Engineer - это специалист, который строит модели машинного обучения, то есть системы, которые создают "магию" интеллектуального поведения

* Это творческая профессия, где каждый случай нужно исследовать

* Мы разобрали, как работает нейронная сеть

* Научились переводить человеческий текст в численное представление (векторы) и объединять их в группы с помощью методов кластеризации

* Все необходимые для работы функции уже реализованы в библиотеках. Мы познакомились с библиотеками gensim, nltk, pymystem, sklearn